In [ ]:
# |default_exp replay_buffer

In [ ]:
# |export
import logging
from functools import cached_property
from queue import Empty, Full
from time import time

import torch
import torch.multiprocessing as mp
from fastcore.basics import store_attr
from fastcore.foundation import L
from torch.utils.data import WeightedRandomSampler

# Replay Buffer
> Buffer that save past experience and allow to sample.

In [ ]:
# |export
class PrioritizedReplayBuffer:
    def __init__(
        self,
        capacity,
        queue_capacity=128,
        prob_alpha=0.5,
        e=1e-5,
        beta=1.0,
        beta_increment=1e-5,
        device="cpu",
    ):
        store_attr()
        self.experience_queue = mp.Queue(queue_capacity)

    def push(self, *args):
        try:
            self.experience_queue.put(args, False)
        except Full:
            logging.warning("Replay buffer queue is full")

    @cached_property
    def buffer(self):
        return L()

    @cached_property
    def pos(self):
        return 0

    @cached_property
    def priorities(self):
        return torch.zeros(
            (self.capacity,), dtype=torch.float, device=self.device, requires_grad=False
        )

    @cached_property
    def last_insert_time(self):
        return time()

    def empty_queue(self):
        try:
            while True:
                self._push(*self.experience_queue.get(False))
                self.last_insert_time = time()
        except Empty:
            pass

    def _push(self, state, next_state, action, reward, done):
        max_prio = self.priorities.max() if self.buffer else 1.0

        if len(self.buffer) < self.capacity:
            self.buffer.append((state, next_state, action, reward, done))
        else:
            self.buffer[self.pos] = (state, next_state, action, reward, done)

        self.priorities[self.pos] = max_prio
        self.pos = (self.pos + 1) % self.capacity

    def sample(self, batch_size):
        self.beta = min(1.0, self.beta + self.beta_increment)

        prios = self.priorities[: len(self)]
        indices = L(WeightedRandomSampler(prios, batch_size, replacement=False))
        samples = self.buffer[indices]

        total = len(self.buffer)
        weights = (total * (prios / prios.sum())[indices]) ** (-self.beta)
        weights /= weights.max()

        return samples, indices, weights

    def update_priorities(self, indices, errors):
        p = torch.nan_to_num(errors, nan=0.0, posinf=0.0, neginf=0.0)
        p = torch.nn.functional.relu(p)
        if not torch.equal(p, errors):
            logging.warn(
                "Received invalid priority, either `inf`, `nan` or element < 0"
            )
        self.priorities[indices] = (p + self.e) ** self.prob_alpha

    def __len__(self):
        return len(self.buffer)

In [ ]:
# |hide
# |eval: false
from nbdev.doclinks import nbdev_export

nbdev_export()